In [1]:
import stan
import nest_asyncio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


nest_asyncio.apply()  # required to run in notebook

# Chapter 6 Exercises

## Exercise 6.12

a. $Y|\pi \sim Bin(20,\pi)$ and $\pi ~ Beta(1,1)$ with $Y = 12$

 
```
model <- 'data {
    int<lower=0, upper=20> Y;
}
parameters {
    real<lower=0, upper =1> pi;
}

model {
    Y ~ binomial(20, pi);
    pi ~ Beta(2,2)
}'



```
